# Naive Paired Verifier Data Collection
One simple way of getting negatives is by shuffling the dataset (mixing answers from other inputs to this one).
We are also trying to gather multiple positives and realistic negatives by sampling from SOTA systems (one prong is in [this repo](https://github.com/matt-seb-ho/reprover_hf)).
However given my compute woes, here is a simple baseline for the time being.

In [4]:
import json
import random
from time import perf_counter
from tqdm import tqdm
# preprocessing script lives in src directory
import sys
sys.path.append("../src")

In [5]:
# preprocessing depends on lean_dojo which has a newer git requirement
import os
path_with_updated_git_binary = "/home/msho/.personalbin/bin"  
os.environ["PATH"] = path_with_updated_git_binary + os.pathsep + os.environ["PATH"]

In [3]:
from prep_sft_data import preprocess_data
from utils import prepend_repo_root, _pp

/mnt/hdd/msho/miniconda3/envs/gfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-06 23:28:38,371	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


[2024-03-06 23:28:40,833] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/hdd/msho/miniconda3/envs/gfn/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/mnt/hdd/msho/miniconda3/envs/gfn/lib/python3.10/site-packages/torch/cuda/__init__.py:651: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [6]:
# RANDOM_TRAIN_DATA_PATH = prepend_repo_root("data/leandojo_benchmark_4/random/train.json")
RANDOM_VAL_DATA_PATH = prepend_repo_root("data/leandojo_benchmark_4/random/val.json")
# NOVELP_TRAIN_DATA_PATH = prepend_repo_root("data/leandojo_benchmark_4/novel_premises/train.json")
# thm_data = preprocess_data(RANDOM_TRAIN_DATA_PATH, True, False)
thm_data = preprocess_data(RANDOM_VAL_DATA_PATH, True, False)

100%|██████████| 2000/2000 [00:00<00:00, 38094.89it/s]
2024-03-06 23:42:11.334 | INFO     | prep_sft_data:preprocess_data:52 - 4641 examples loaded


In [7]:
def add_random_negatives(data, output_file=None, max_iters=1000, seed=42):
    random.seed(seed)
    pairs = []
    for entry in tqdm(data, total=len(data)):
        positive = entry["tactic"]
        negative = None
        for _ in range(max_iters):
            candidate = random.choice(data)["tactic"]
            if candidate != positive:
                negative = candidate
                break
        if negative is None:
            continue
        pairs.append({
            "state": entry["state"],
            "positive": positive,
            "negative": negative
        })

    if output_file:
        with open(output_file, 'w') as f:
            output = {"seed": seed, "pairs": pairs}
            json.dump(output, f)
            print(f"Wrote paired data to: {output_file}")
    
    return pairs

In [8]:
pairs = add_random_negatives(
    thm_data,
    output_file=prepend_repo_root("data/paired_random_val.json")
)

100%|██████████| 4641/4641 [00:00<00:00, 351856.64it/s]

Wrote paired data to: /mnt/hdd/msho/gfn_ntp/data/paired_random_val.json


In [21]:
dummy = random.choice(pairs)

In [22]:
print(dummy["state"])

case refl
l : List Char
c : Char
r : List Char
h : ValidFor l (c :: r) { s := { data := List.reverse l ++ c :: r }, i := { byteIdx := utf8Len (List.reverse l) } }
⊢ ValidFor (c :: l) r { s := { data := List.reverseAux l (c :: r) }, i := { byteIdx := utf8Len l + csize c } }


In [25]:
print(dummy["positive"])

constructor


In [26]:
print(dummy["negative"])

have h2_meas : MeasureTheory.AEStronglyMeasurable (fun y : G => ∫ x : G, ‖L (f y) (g (x - y))‖ ∂μ) ν := h_meas.prod_swap.norm.integral_prod_right'
